In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import json
import Levenshtein

import utils

In [ ]:
spotify = utils.auth()

In [ ]:
track_limit = 5
festival_artist_file = 'artist_lists/heatwave_artists.txt'
playlist_title = f'Heatwave (top {track_limit})'

# Finding song overlap

In [ ]:
my_songs_sorted = utils.get_my_playlist_songs(spotify)

In [ ]:
artists_with_songs = {}
for song in my_songs_sorted:
    for a in song["artists"]:
        artist = a['name']
        if artist in artists_with_songs:
            artists_with_songs[artist].append(song["name"])
        else:
            artists_with_songs[artist] = [song["name"]]
my_artists = list(artists_with_songs.keys())

In [ ]:
artists_with_songs

In [ ]:
def check_str_in_list(s, ls, max_dist=1):
    matches = []
    for _s in ls:
        if Levenshtein.distance(s, _s, processor=str.upper) <= max_dist:
            matches.append(_s)
    return matches

In [ ]:
festival_artists = []
with open(festival_artist_file) as f:
    lines = f.readlines()
    for line in lines:
        if line[-1:] == "\n":
            line = line[:-1]
        if line == "":
            continue
        festival_artists.append(line)

In [ ]:
for artist in festival_artists:
    matches = check_str_in_list(artist, my_artists, max_dist=1)
    if matches != []:
        print(artist)
        for m in matches:
            print("\t", m)
            for song in artists_with_songs[m]:
                print("\t\t", song)

# Finding top tracks of festival artists

In [ ]:
def get_top_tracks(artist_name, track_limit=10):
    if artist_name[:2] == "??":
        print(f"{artist_name[2:]} not on Spotify - skipped")
        return None
    print(f"search for: {artist_name}")
    artist_result = spotify.search(q=f"artist:{artist_name}", type="artist")
    if len(artist_result['artists']['items']) == 0:
        print('NO MATCH FOUND')
        return None
    best_match = artist_result["artists"]["items"][0]
    print(f"\tbest artist match: {best_match['name']}")
    artist_uri = best_match["uri"]
    tracks = spotify.artist_top_tracks(artist_uri)["tracks"][:track_limit]
    track_info = {track["name"]: track["uri"] for track in tracks}
    return track_info

In [ ]:
festival_artist_tracks = {}
for artist_name in festival_artists:
    track_info = get_top_tracks(artist_name, track_limit)
    if track_info is not None:
        festival_artist_tracks[artist_name] = track_info

In [ ]:
playlist = spotify.user_playlist_create(spotify.me()["id"], playlist_title)

In [ ]:
for artist_name, track_info in festival_artist_tracks.items():
    print(f"adding tracks from {artist_name}")
    track_uris = list(track_info.values())
    try:
        spotify.playlist_add_items(playlist['id'], track_uris)
        for track_name in track_info.keys():
            print(f"\tadded track {track_name}")
    except:
        print(f"some error adding songs from {artist_name}")